In [0]:
import pandas as pd

df1 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2020_Present.csv")
df2 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2010_2019.csv")
df3 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2000_2009.csv")
df4 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_1991_1999.csv")
df5 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_2010_Present.csv")
df6 = pd.read_csv("/Workspace/Users/jhw7598@uncw.edu/LOB_Pracitcum_Datasets/foia_1991_2009.csv")

In [0]:
merged_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

merged_df

In [0]:
#Dropping all categorical columns
merged_df = merged_df.drop(columns=[col for col in merged_df.select_dtypes('object').columns if col != 'LoanStatus'])

merged_df.head()

In [0]:
merged_df=merged_df.apply(
    lambda x: x.fillna('Unknown') if x.dtype == 'object' else x.fillna(x.median())
)

In [0]:
merged_df.isnull().sum()

In [0]:
merged_model = merged_df[merged_df['LoanStatus'].isin(['CHGOFF','PIF'])].copy()
merged_model['loan_default'] = merged_model['LoanStatus'].apply(lambda x: 1 if x=='CHGOFF' else 0)

In [0]:
merged_model['loan_default'].value_counts()

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [0]:
X = merged_model.drop(columns=['LoanStatus','loan_default','GrossChargeoffAmount'])
y = merged_model['loan_default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .3, random_state=42, stratify=y)

In [0]:
rf = RandomForestClassifier(
    class_weight='balanced', #helps with minority class
    n_estimators=200, #each tree sees diff sample then combines predictions for majority vote/ more trees make more stable avg out random noise 
    max_depth=15, #limits how deep tree can grow to prevent overfitting/ too deep will memorize training data
    min_samples_leaf=50, #Prevents overly specific leafs (finals nodes) from being created
    random_state=42
)
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

# Evaluate
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [0]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [0]:
y_train_pred = rf.predict(X_train)
from sklearn.metrics import classification_report

print(classification_report(y_train, y_train_pred))


In [0]:
import pandas as pd
import matplotlib.pyplot as plt

feature_importances = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importances.sort_values().plot(kind='barh', figsize=(8,6))
plt.title('Feature Importance')
plt.show()


In [0]:
from sklearn.metrics import RocCurveDisplay
#VERY GOOD AT PREDICTING ACTUAL DEFAULTS 
#Get predicted probabilities for class 1
y_proba = rf.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba, name="Random Forest")
